In [ ]:
import pandas as pd


### get the penn_df_urls

In [ ]:
# json export of db
wwe_df = pd.read_json('./configs/wwe_df.json') # <-- json export of AWS Athena results

# path to repo decker deploy script
deploy_script_path = './deploy.sh' # <-- Make sure to edit for your docker mount -v

# WWE IG Username (must match `username` in wwe_df!)
input_wwe_username = 'thelanawwe' # <-- kayfabe!

### Loop through urls for BlackIvyStories IG username ...


In [3]:
import pandas as pd

In [11]:
ig_df = pd.read_csv('./data/blackivystories_PostsExtractor_03012021.csv')

In [4]:
df = pd.read_json('./data/pennivystories.json')

In [5]:
ivy_input_name = 'penn'

In [9]:
df.columns

Index(['language', 'text_angle', 'orientation', 'regions', 'postId', 'txt',
       'ivy'],
      dtype='object')

In [33]:
def getUrl(x,igx):

    url = igx[igx.postId == int(x.postId)].postUrl
    
    return url

In [34]:
df['url'] = df.apply(lambda x: getUrl(x,ig_df), axis=1)

ValueError: Wrong number of items passed 0, placement implies 1

In [25]:
pid

2393808657625551360

In [18]:
ig_df.postId

0      2435456592628756077
1      2435456592645585531
2      2435456592620377387
3      2433745246216486181
4      2433745246199538105
              ...         
623    2332251841385804740
624    2332224388852583995
625    2332164025536572487
626    2332161255416871592
627    2332142997762728939
Name: postId, Length: 628, dtype: int64

In [12]:
for pid in df[df.ivy == ivy_input_name].postId.unique():
    print(pid)

2374422979850657792
2358282383087974912
2343993744749285376
2341764308463485440
2342541965549375488
2355640402197289472
2370106296235291136
2375200258251882496
2431454867161038848
2365805455579938816
2340274376262271488
2351078056535642624
2371509661070601728
2371339433657387520
2369448237762686464
2344748250403628544
2377111197339474944
2338222891854142976
2357090857364396544
2369962037578467840
2353425698519369728
2368646388541347840
2338983808585733632
2423676819573441536
2370857024641835008
2369868362940916736
2360017524659430912
2351991123658340864
2387436188601923072
2337245180423726592
2359224950042398208
2370069315149207552
2345503318665021440
2379288361677688320
2380882158408921088
2370069315132579840
2368606957461770752
2363363966698570752
2361479988018189312
2365650660537718272
2369329806791083008
2372085074314565120
2361479988001610752
2346972879054670848
2372835206844879872
2349823662758363648
2354672320117003264
2362839721404805120
2426535005074640384
2332161255416871424


In [6]:
for url in df[df.ivy == ivy_input_name].postUrl.unique():
  
    ##################################################################
    # Clean up URL (in case lacking https or has trailing slash)...
    if not url.startswith('https://'):
        url = 'https://' + url
    if url[-1] =='/':
        url = url[:-1]
    
    ##################################################################
    # Prepare Dockerfile (using template to overwrite for next URL)
    dockerfile_in = open("./configs/dockerfile_template", "rt")
    dockerfile_out = open("./dockerfile", "wt")
    # Replace the value w/ URL -- this loops through lines, would be better using regex...
    for line in dockerfile_in:
        # This is how url gets passed to container...
        dockerfile_out.write(line.replace('replace_with_url', '"{}"'.format(url)))
    #close input and output dockerfile & template
    dockerfile_in.close()
    dockerfile_out.close()
    
    ##################################################################
    # BEGIN LOOPING THROUGH URLS BY LAUNCHING DOCKER CommentGetters!
    print('Getting comments for IG post --> {}'.format(url))
    try:
        !bash $deploy_script_path
    except:
        print('oops! failed for {}'.format(url))
        break

AttributeError: 'DataFrame' object has no attribute 'postUrl'